# Problem formulation

![cargo](figs/cargo.png)
# State
$x = \begin{bmatrix} p & v & z \end{bmatrix}^T \in \R^3$
# Control input 
$u \in \R^1$
# Dynamics
$\dot x  = \underbrace{\begin{bmatrix} v\\ -\frac{F_r(v)}{m} \\ v_0 - v\end{bmatrix}}_{f(x)} + \underbrace{\begin{bmatrix} 0\\ \frac{1}{m} \\ 0\end{bmatrix} u}_{g(x)}$
# Input constraints:
$$u_{min}<u<u_{max}$$
$$v_{min}<v<v_{max}$$

# Stability objective:
$$v \rightarrow v_d$$

# Safety objective:
$z \geq T_h v$ ($T_h$ - lookahead time)



# System constraints 


The problem can be solved via CLF and CBF, such that CLF will tend the system to equilibrium $v \rightarrow v_d$ and CBF to $z \geq T_h v$.

# Design of CLF
1. The Lyapunov function $V(s) = 0$ when $s=s_d$ and $V(s) > 0 \forall s \in \R^n \ {s_d}$.
2. $\Omega_c := \{ s \in \R^n : V(x) \leq c\}$, a sublevel set is bounded (sublevel since QP problem find an elliptical set.)
3. $\dot V(x, u) < -\gamma V(x)$
   
To reach the condition, 
Then $V(x) = (v - v_d)^2 $

$\nabla V(x) = \begin{bmatrix} 0 \\ 2(v-v_d) \\ 0 \end{bmatrix}$

$L_f V(x) = -\frac{2 F_r(v)}{m}(v - v_d)$, $L_g V(x) = \frac{2}{m}(v-v_d)$

$\dot V(x) + \lambda V(x) = L_f V(x) + L_g V(x) u + \lambda V(x) = (v - v_d)\{ \frac{2}{m}(u - F_r) + \lambda (v - v_d)\}\leq0$

The value of rolling resistance $F_r$ is very small.

- If $v < v_d$ and $u >> F_r$, the term $\{ \frac{2}{m}(u - F_r) + \lambda (v - v_d)\}$ [1] will be positive and the constraints will be satisfied.
- If $v > v_d$, the term [1] will be negative if the cargo will decelerate enough $u < F_r$. 

Therefore the function V in a valid CLF (if $\lambda$ is small enough).

# Design CBF
Safety objective: $z \geq T_h v$ ($T_h$ - lookahead time)

$\nabla h(x) = \begin{bmatrix} 0 \\ -T_h \\1 \end{bmatrix}$

$L_f h(x) = (F_r(v) - u)\frac{T_h}{m} + (v_0 - v)$, $L_g h(x) = \frac{-T_h}{m}$

$\dot h(x) + \gamma h(x) = (F_r(v) - u)\frac{T_h}{m} + (v_0 - v) +\gamma \frac{-T_h}{m}\geq 0$



$\dot h(x) + \gamma h(x) = T_h c_dg + v_0 + \gamma z - (1+ T_h \gamma)v$


When the value of of v is big compared to the positive terms, the terms might not be satisfied.
To avoid it, we modify a CBF to include the term regarding a minimum braking distance required to decelerate from $v$ to $v_0$:

$h(x) = z - T_h v - \frac{1}{2}\frac{(v-v_0)^2}{c_d g}$

Then we get $\dot h(x) = \frac{1}{m}(T_h + \frac{v-v_0}{c_d g})(F_r(v) - u) + (v_0 - v)$

Under maximum deceleration ($u = -c_dmg$): $\dot h(x) = \frac{1}{m}T_hF_r(v) + T_hc_dg \geq 0$

Therefore, the constraint is always feasible at any state, so $h(x)$ is a valid CBF.


In [1]:
import cvxpy as cp
import mujoco
import mediapy as media
import numpy as np
import casadi as cs

from robot_descriptions.loaders.mujoco import load_robot_description
from darli.robots import manipulator
from darli.modeling import Robot
from darli.backend import PinocchioBackend

%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


In [23]:
F_r = cp.Variable()
T_h = cp.Variable()
# m = cp.()

v_0 = cp.Variable()
v_d = cp.Variable()

v = cp.Parameter(1, "v")
p = cp.Parameter(1, "p")
z = cp.Parameter(1, "z")

v_dot = cp.Parameter(1, "v_dot")
u = cp.Parameter(3, "u")

c_d = 0.3
m = 10

g = 9.8
u_min = np.array([-m*c_d*g]*3)
u_max = np.array([m*c_d*g]*3)


In [24]:

des_v = 10

A = cp.Parameter(3, "A")
B = cp.Parameter(3, "B")

q = cp.Parameter(3, "q")
q_dot = cp.Parameter(3, "q_dot")

In [25]:
def rolling_resistance(v, fs=[1, 1, 1]):
    return fs[0] + fs[1]*v + fs[2]*v**2

def lead_vehicle_velocity(t):
    return des_v * np.sin(0.1 * t)

In [26]:
cbf = z - T_h*v - (v - v_0)**2/(2* c_d * g)
clf = (v - v_d)**2
cost= cp.sum_squares(p) + cp.sum_squares(u)

constraints = [cbf>=0, clf>=0, u_min <= u, u <= u_max, cp.multiply(A, q_dot)==cp.multiply(B, u)]
#v_dot*m + F_r ==u] #, v_var <= v_max ]

In [27]:
objective = cp.Minimize(cost)
problem = cp.Problem(objective, constraints)

In [30]:
qpos = np.zeros(3)
qvel = np.zeros(3)
dt = 0.1
# F_r.value = 1
F_r.value = rolling_resistance(0)
# m.value = 10
v_0.value = lead_vehicle_velocity(1)
T_h.value = dt
v_d.value = 10
z.value = (v_0.value - qvel[1])*dt
p.value = qpos
v.value = qvel
c_d.value = 0.3

g = 9.8
# u_min.value = -m*c_d*g
# u_max.value = m*c_d*g

problem.solve()
problem.solution()

ValueError: Invalid dimensions () for Parameter value.

In [ ]:
t_end = 10
duration = t_end  # (seconds)
framerate = 30  # used just for visualization

# Simulate and display video.
frames = []
ts = []
Kp, Kd = 100, 10
circle_r = 0.1
state_history = []
control_history = []
task_history = []
error_history = []

dt_control = 1e-2
t_next = dt_control

u_min_val = -4
u_max_val = 4 

v_dot_min_val = -10
v_dot_max_val = 10

v_max_val = 0.2

# z_pos = model.bodies["ee"].jacobian.world_aligned @ trajectory(0, circle_r)
z_pos = 0
traj_frequency = 0.1

qpos = np.zeros(3)
qvel = np.zeros(3)
time = 0
while time < duration:
    q, v = (
        qpos,
        qvel,
    )
    state = np.hstack((q, v))
    
    pos = model.bodies["ee"].position
    jac_curr = model.bodies["ee"].jacobian.world_aligned
    jac_dt_curr = model.bodies["ee"].jacobian_dt.world_aligned

    M_curr = model.inertia(q)
    h_curr = model.coriolis(q, v) + model.gravity(q)

    traj_ddt_curr = trajectory_ddt(data.time, circle_r, traj_frequency)
    traj_dt_curr = trajectory_dt(data.time, circle_r, traj_frequency)
    traj_curr = trajectory(data.time, circle_r, np.array([0, 0.0, 0.2]), traj_frequency)
    v_var.value = v
    J_traj.value = jac_curr[:3]

    b_traj.value = (
        # (jac_dt_curr @ v)[:3]
        # - traj_ddt_curr
        + Kd * ((jac_curr @ v)[:3] - traj_dt_curr)
        + Kp * (pos - traj_curr)
    )

    u_min.value = u_min_val
    u_max.value = u_max_val

    v_dot_min.value = v_dot_min_val
    v_dot_max.value = v_dot_max_val

    v_max.value = v_max_val
    
    M.value = np.array(M_curr)
    h.value = np.array(h_curr)

    B.value = np.eye(nu)

    problem.solve()
    problem.solution()
    data.ctrl = np.array(u.value)  # .reshape(mj_model.nu, )
    state_history.append(pos.copy())
    task_history.append(traj_curr)
    error_history.append(pos - traj_curr)
    ts.append(data.time)
    control_history.append(data.ctrl.copy())

    mujoco.mj_step(mj_model, data)
    
    time+=dt_control